In [1]:
# 기본라이브러리 import
#from google.colab import drive
#import os, json, pickle
#import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

#from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# 파이토치 라이브러리 import
# import torch
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# from torch.utils.data import random_split

#Keras Import
import tensorflow as tf
from tensorflow.keras import layers
from datetime import timedelta, datetime

# 데이터 정규화
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

import ipaddress

# 구글 드라이브 mount
#drive.mount('/content/gdrive')

# 데이터 파일 위치
#C:\Users\mariu\내 드라이브\Colab Notebooks\Network
# colab_path = 'C:/Documents/VM/disk/notebook/'
# colab_write_path = "C:/Users/mariu/Downloads/"

In [2]:

df = pd.read_csv("NF-UQ-NIDS-total.csv",index_col=False)


In [3]:
# 확장판에서 사용
#cols = df.columns.drop(['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT', 'PROTOCOL', 'L7_PROTO', 'DNS_QUERY_ID','Label','Attack','Dataset'])  #명목형변수, 레이블 변수 제외
#cols = df.columns.drop(['IPV4_SRC_ADDR_1','IPV4_SRC_ADDR_2','Label','Attack','Dataset', 'Label_1','Attack_1','Dataset_1', 'Label_2','Attack_2','Dataset_2'])  #명목형변수, 레이블 변수 제외
cols = df.columns.drop(['Label','Attack'])  #명목형변수, 레이블 변수 제외#cols = df.columns.drop(['Attack','Dataset'])  #명목형변수, 레이블 변수 제외
X = df[cols]
dummies = pd.get_dummies(df['Attack']) # Classification
outcomes = dummies.columns
num_classes = len(outcomes)
Y = dummies.values

#==================
dummiesLabel = pd.get_dummies(df['Label']) # Classification
y_1_label = df['Label'].values
y_label = dummiesLabel.values

#import pandas as pd
#import numpy as np

def extend_sparse(val):
    if val in ['Analysis', 'Exploits', 'Fuzzers', 'Shellcode', 'Theft', 'Worms', 'mitm']: return 1
    return 0

y_1_attack = pd.DataFrame(df['Attack'])

is_sparse = y_1_attack.applymap(extend_sparse)
y_1_enforce = is_sparse.values
y_enforce = pd.get_dummies(is_sparse['Attack']) # Classification

#=======================

def max_8G(val):
    if (val > 1.0e+9): return 1.0e+9
    return val

#cols =['DST_TO_SRC_SECOND_BYTES', 'SRC_TO_DST_SECOND_BYTES', 'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT']
cols =['DST_TO_SRC_SECOND_BYTES', 'SRC_TO_DST_SECOND_BYTES', 'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT',
'DST_TO_SRC_SECOND_BYTES_1', 'SRC_TO_DST_SECOND_BYTES_1', 'SRC_TO_DST_AVG_THROUGHPUT_1', 'DST_TO_SRC_AVG_THROUGHPUT_1',
'DST_TO_SRC_SECOND_BYTES_2', 'SRC_TO_DST_SECOND_BYTES_2', 'SRC_TO_DST_AVG_THROUGHPUT_2', 'DST_TO_SRC_AVG_THROUGHPUT_2'] 

X[cols] = X[cols].applymap(max_8G)

In [4]:
# 데이터 정규화
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV


#scaler_train_minmax = preprocessing.MinMaxScaler()

scaler_train = preprocessing.StandardScaler()
#scaler_train = preprocessing.MinMaxScaler()
scaler_train = scaler_train.fit(X)
X = pd.DataFrame(scaler_train.transform(X),index=np.arange(0,X.shape[0],1), columns = X.columns)
#X = scaler_train.transform(X)


In [5]:
def max_3sigma(val):
    if (val < -3): return -3
    if (val > 3): return 3
    return val

X = X.applymap(max_3sigma)

In [6]:
y1 = tf.argmax(dummies, axis=1)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, shuffle = True, random_state=33)
x_train_label, x_test_label, y_train_label, y_test_label = train_test_split(X, y_label, test_size=0.20, shuffle = True, random_state=33)
x_train_enforce, x_test_enforce, y_train_enforce, y_test_enforce = train_test_split(X, y_enforce, test_size=0.20, shuffle = True, random_state=33)
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(X, y1.numpy(), test_size=0.20, shuffle = True, random_state=33)

2023-10-09 23:29:45.527695: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
#model1
#initializer = "glorot_uniform"
initializer = tf.keras.initializers.GlorotUniform(seed=33)
constraints = None


model_1 = tf.keras.Sequential([
  tf.keras.layers.Dense(64, input_dim=X.shape[1], kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints, activation='relu',name='dense_1a'),
  tf.keras.layers.Dense(128, input_dim=X.shape[1], kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints,activation='relu',name='dense_1b'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(128, input_dim=X.shape[1], kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints,activation='relu',name='dense_1c'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(64, input_dim=X.shape[1], kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints,activation='relu',name='dense_1d'),
  tf.keras.layers.Dense(Y.shape[1],activation='softmax')
  ])
'''
model_1 = tf.keras.Sequential([
  tf.keras.layers.Dense(64, input_dim=X.shape[1], kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints, activation='relu'),
  tf.keras.layers.Dense(128, input_dim=64, kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints,activation='relu'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(128, input_dim=128, kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints,activation='relu'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(64, input_dim=128, kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints,activation='relu'),
  tf.keras.layers.Dense(Y.shape[1],input_dim=64, activation='softmax')
  ])
'''

'\nmodel_1 = tf.keras.Sequential([\n  tf.keras.layers.Dense(64, input_dim=X.shape[1], kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints, activation=\'relu\'),\n  tf.keras.layers.Dense(128, input_dim=64, kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints,activation=\'relu\'),\n  tf.keras.layers.Dropout(0.3),\n  tf.keras.layers.Dense(128, input_dim=128, kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints,activation=\'relu\'),\n  tf.keras.layers.Dropout(0.3),\n  tf.keras.layers.Dense(64, input_dim=128, kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints,activation=\'relu\'),\n  tf.keras.layers.Dense(Y.shape[1],input_dim=64, activation=\'softmax\')\n  ])\n'

In [8]:
#model 1
model_1.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adam')

log_dir_1 = datetime.now().strftime("%Y%m%d%H%M%S")
print(log_dir_1)
monitor_1 = [
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=8, verbose=1, mode='auto',restore_best_weights=True),
#             tf.keras.callbacks.TensorBoard(log_dir=log_dir_1, histogram_freq=1)
#             tf.keras.callbacks.TensorBoard(histogram_freq=1)
            ]
history_1  = model_1.fit(x_train,y_train,validation_data=(x_test,y_test), verbose=2, batch_size=1024, epochs=100)
#history_1  = model_1.fit(x_train,y_train,validation_split=0.2, callbacks=[monitor_1],verbose=2, batch_size=1024, epochs=100)


20231009232948
Epoch 1/100
852/852 - 5s - loss: 0.2976 - accuracy: 0.9120 - val_loss: 0.1299 - val_accuracy: 0.9641 - 5s/epoch - 5ms/step
Epoch 2/100
852/852 - 4s - loss: 0.1294 - accuracy: 0.9653 - val_loss: 0.1030 - val_accuracy: 0.9719 - 4s/epoch - 4ms/step
Epoch 3/100
852/852 - 4s - loss: 0.1103 - accuracy: 0.9706 - val_loss: 0.0949 - val_accuracy: 0.9750 - 4s/epoch - 4ms/step
Epoch 4/100
852/852 - 4s - loss: 0.1008 - accuracy: 0.9731 - val_loss: 0.0884 - val_accuracy: 0.9768 - 4s/epoch - 4ms/step
Epoch 5/100
852/852 - 4s - loss: 0.0954 - accuracy: 0.9748 - val_loss: 0.0845 - val_accuracy: 0.9774 - 4s/epoch - 4ms/step
Epoch 6/100
852/852 - 4s - loss: 0.0908 - accuracy: 0.9760 - val_loss: 0.0811 - val_accuracy: 0.9786 - 4s/epoch - 4ms/step
Epoch 7/100
852/852 - 4s - loss: 0.0880 - accuracy: 0.9767 - val_loss: 0.0804 - val_accuracy: 0.9788 - 4s/epoch - 4ms/step
Epoch 8/100
852/852 - 4s - loss: 0.0850 - accuracy: 0.9772 - val_loss: 0.0802 - val_accuracy: 0.9789 - 4s/epoch - 4ms/step
E

In [9]:
#Training  model 2  with different structure
initializer = tf.keras.initializers.GlorotUniform(seed=64)

model_2 = tf.keras.Sequential([
      tf.keras.layers.Dense(64, input_dim=X.shape[1], kernel_initializer=initializer, activation='relu',name='dense_2a'),
      tf.keras.layers.Dense(128, input_dim=64, kernel_initializer=initializer, activation='relu',name='dense_2b'),
      tf.keras.layers.Dropout(0.3),
      tf.keras.layers.Dense(256, input_dim=128, kernel_initializer=initializer, activation='relu',name='dense_2c'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(128, input_dim=256, kernel_initializer=initializer, activation='relu',name='dense_2d'),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Dense(Y.shape[1],activation='softmax')
      ])

In [10]:
#from tensorflow.keras.callbacks import EarlyStopping

model_2.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adam')

log_dir_2 = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

#monitor_2 = [
#             tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=8, verbose=1, mode='auto',restore_best_weights=True),
#             tf.keras.callbacks.TensorBoard(log_dir=log_dir_2, histogram_freq=1)
#            ]
#history_2  = model_2.fit(x_train,y_train,validation_data=(x_test,y_test), callbacks=[monitor_2],verbose=2, batch_size=512, epochs=100)
history_2  = model_2.fit(x_train,y_train,validation_data=(x_test,y_test), verbose=2, batch_size=512, epochs=100)

Epoch 1/100
1704/1704 - 8s - loss: 0.2040 - accuracy: 0.9397 - val_loss: 0.1090 - val_accuracy: 0.9701 - 8s/epoch - 5ms/step
Epoch 2/100
1704/1704 - 7s - loss: 0.1112 - accuracy: 0.9700 - val_loss: 0.0914 - val_accuracy: 0.9750 - 7s/epoch - 4ms/step
Epoch 3/100
1704/1704 - 7s - loss: 0.0984 - accuracy: 0.9735 - val_loss: 0.0859 - val_accuracy: 0.9776 - 7s/epoch - 4ms/step
Epoch 4/100
1704/1704 - 7s - loss: 0.0908 - accuracy: 0.9755 - val_loss: 0.0818 - val_accuracy: 0.9776 - 7s/epoch - 4ms/step
Epoch 5/100
1704/1704 - 7s - loss: 0.0866 - accuracy: 0.9767 - val_loss: 0.0788 - val_accuracy: 0.9790 - 7s/epoch - 4ms/step
Epoch 6/100
1704/1704 - 7s - loss: 0.0833 - accuracy: 0.9775 - val_loss: 0.0762 - val_accuracy: 0.9795 - 7s/epoch - 4ms/step
Epoch 7/100
1704/1704 - 7s - loss: 0.0811 - accuracy: 0.9782 - val_loss: 0.0767 - val_accuracy: 0.9797 - 7s/epoch - 4ms/step
Epoch 8/100
1704/1704 - 7s - loss: 0.0798 - accuracy: 0.9784 - val_loss: 0.0736 - val_accuracy: 0.9803 - 7s/epoch - 4ms/step


In [11]:
#3 rd model for stack
model_3 = tf.keras.Sequential([
      tf.keras.layers.Dense(512, input_dim=X.shape[1], kernel_initializer=initializer, activation='relu',name='dense_3a'),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Dense(256, input_dim=X.shape[1], kernel_initializer=initializer, activation='relu',name='dense_3b'),
      tf.keras.layers.Dropout(0.3),
      tf.keras.layers.Dense(128, input_dim=X.shape[1], kernel_initializer=initializer, activation='relu',name='dense_3c'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(128, input_dim=X.shape[1], kernel_initializer=initializer, activation='relu',name='dense_3d'),
      tf.keras.layers.Dense(Y.shape[1],activation='softmax')
      ])

In [12]:

model_3.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adam')

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

monitor_3 = [
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=8, verbose=2, mode='auto',restore_best_weights=True),
             tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
            ]
history_3  = model_3.fit(x_train,y_train,validation_data=(x_test,y_test), callbacks=[monitor_3],verbose=2, batch_size=512, epochs=100)

Epoch 1/100
1704/1704 - 13s - loss: 0.1724 - accuracy: 0.9497 - val_loss: 0.0970 - val_accuracy: 0.9744 - 13s/epoch - 8ms/step
Epoch 2/100
1704/1704 - 12s - loss: 0.1005 - accuracy: 0.9730 - val_loss: 0.0863 - val_accuracy: 0.9772 - 12s/epoch - 7ms/step
Epoch 3/100
1704/1704 - 12s - loss: 0.0900 - accuracy: 0.9760 - val_loss: 0.0838 - val_accuracy: 0.9771 - 12s/epoch - 7ms/step
Epoch 4/100
1704/1704 - 12s - loss: 0.0853 - accuracy: 0.9772 - val_loss: 0.0790 - val_accuracy: 0.9793 - 12s/epoch - 7ms/step
Epoch 5/100
1704/1704 - 12s - loss: 0.0817 - accuracy: 0.9781 - val_loss: 0.0758 - val_accuracy: 0.9798 - 12s/epoch - 7ms/step
Epoch 6/100
1704/1704 - 12s - loss: 0.0790 - accuracy: 0.9787 - val_loss: 0.0744 - val_accuracy: 0.9800 - 12s/epoch - 7ms/step
Epoch 7/100
1704/1704 - 12s - loss: 0.0775 - accuracy: 0.9791 - val_loss: 0.0735 - val_accuracy: 0.9805 - 12s/epoch - 7ms/step
Epoch 8/100
1704/1704 - 12s - loss: 0.0759 - accuracy: 0.9795 - val_loss: 0.0715 - val_accuracy: 0.9807 - 12s/e

In [13]:
# #4 rd model for stack
# model_4 = tf.keras.Sequential([
#       tf.keras.layers.Dense(128, input_dim=x_train.shape[1], kernel_initializer=initializer, activation='relu',name='dense_4a'),
#       tf.keras.layers.Dense(64, input_dim=x_train.shape[1], kernel_initializer=initializer, activation='relu',name='dense_4b'),
#       tf.keras.layers.Dropout(0.3),
#       tf.keras.layers.Dense(32, input_dim=x_train.shape[1], kernel_initializer=initializer, activation='relu',name='dense_4c'),
#       tf.keras.layers.Dropout(0.2),
#       tf.keras.layers.Dense(128, input_dim=x_train.shape[1], kernel_initializer=initializer, activation='relu',name='dense_4d'),
#       tf.keras.layers.Dense(y_train.shape[1],activation='softmax')
#       ])

In [14]:
# model_4.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adam')

# log_dir_4 = colab_write_path + "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

# monitor_4 = [
#             tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=100, verbose=1, mode='auto',restore_best_weights=True),
#             tf.keras.callbacks.TensorBoard(log_dir=log_dir_4, histogram_freq=1)
#         ]
# history_4  = model_4.fit(x_train,y_train,validation_data=(x_test,y_test), callbacks=[monitor_4],verbose=2, batch_size=512, epochs=100)

In [16]:
model_1.save('model1_1.h5')
model_2.save('model1_2.h5')
model_3.save('model1_3.h5')


In [17]:
from collections import Counter
#type(y_train)
ydp= pd.DataFrame(y_train)
ydp = tf.argmax(ydp, axis=1)
#ydp = pd.DataFrame(ydp)
#ydp.value_counts
dicts = Counter(ydp.numpy())
dicts



Counter({6: 205290,
         2: 288786,
         5: 249311,
         20: 28278,
         19: 43885,
         11: 29889,
         17: 13051,
         3: 1639,
         15: 7863,
         4: 1436,
         10: 1354,
         8: 256,
         9: 185,
         7: 373,
         18: 31,
         1: 214,
         13: 17,
         16: 86,
         0: 38,
         12: 13,
         14: 4})

In [18]:
#from imblearn.over_sampling import SMOTE
#smote = SMOTE(random_state=42)
#X_train_over,y_train_over = smote.fit_resample(x_train,y_train)
MININUM_SAMPLES = 30000
MAXINUM_SAMPLES = 300000

sample_dict = {6: 205290,
         2: 288786,
         5: 249311,
         20: 28278,
         19: 43885,
         11: 29889,
         17: 13051,
         3: 1639,
         15: 7863,
         4: 1436,
         10: 1354,
         8: 256,
         9: 185,
         7: 373,
         18: 31,
         1: 214,
         13: 17,
         16: 86,
         0: 38,
         12: 13,
         14: 4}

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

#ros = RandomOverSampler(random_state=33)
for i in range(21):
    if sample_dict[i] >  MAXINUM_SAMPLES:
        sample_dict[i] =  MAXINUM_SAMPLES

ros = RandomUnderSampler(sampling_strategy = sample_dict, random_state=33)
X_train_under, y_train_under = ros.fit_resample(x_train, y_train)

for i in range(21):
    if sample_dict[i] <  MININUM_SAMPLES:
        sample_dict[i] =  MININUM_SAMPLES

ros = RandomOverSampler(sampling_strategy = sample_dict, random_state=33)
X_train_over, y_train_over = ros.fit_resample(X_train_under, y_train_under)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', x_train.shape, y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (871999, 127) (871999, 21)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (1297272, 127) (1297272, 21)


In [19]:
#6 rd model for stack
initializer = tf.keras.initializers.GlorotUniform(seed=33)
constraints = None

#6 rd model for stack
model_6 = tf.keras.Sequential([
      tf.keras.layers.Dense(256, input_dim=X_train_over.shape[1], kernel_initializer=initializer, activation='relu',name='dense_6a'),
      tf.keras.layers.Dropout(0.3),
      tf.keras.layers.Dense(128, input_dim=X_train_over.shape[1], kernel_initializer=initializer, activation='relu',name='dense_6b'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(128, input_dim=X_train_over.shape[1], kernel_initializer=initializer, activation='relu',name='dense_6c'),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Dense(64, input_dim=X_train_over.shape[1], kernel_initializer=initializer, activation='relu',name='dense_6d'),
      tf.keras.layers.Dense(y_train_over.shape[1],activation='softmax')
      ])

In [21]:
model_6.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adam')

log_dir_6 = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

monitor_6 = [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-7, patience=100, verbose=1, mode='auto',restore_best_weights=True),
            tf.keras.callbacks.TensorBoard(log_dir=log_dir_6, histogram_freq=1)
        ]
#print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', x_test.shape, y_test.shape)
#print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
history_6  = model_6.fit(X_train_over,y_train_over,validation_data=(x_test,y_test), callbacks=[monitor_6],verbose=2, batch_size=512, epochs=100)

Epoch 1/100
2534/2534 - 11s - loss: 0.2983 - accuracy: 0.9011 - val_loss: 0.1333 - val_accuracy: 0.9634 - 11s/epoch - 4ms/step
Epoch 2/100
2534/2534 - 10s - loss: 0.1593 - accuracy: 0.9503 - val_loss: 0.1111 - val_accuracy: 0.9722 - 10s/epoch - 4ms/step
Epoch 3/100
2534/2534 - 10s - loss: 0.1382 - accuracy: 0.9571 - val_loss: 0.1108 - val_accuracy: 0.9728 - 10s/epoch - 4ms/step
Epoch 4/100
2534/2534 - 10s - loss: 0.1284 - accuracy: 0.9598 - val_loss: 0.1142 - val_accuracy: 0.9702 - 10s/epoch - 4ms/step
Epoch 5/100
2534/2534 - 10s - loss: 0.1228 - accuracy: 0.9615 - val_loss: 0.0998 - val_accuracy: 0.9758 - 10s/epoch - 4ms/step
Epoch 6/100
2534/2534 - 10s - loss: 0.1187 - accuracy: 0.9627 - val_loss: 0.0997 - val_accuracy: 0.9754 - 10s/epoch - 4ms/step
Epoch 7/100
2534/2534 - 10s - loss: 0.1154 - accuracy: 0.9635 - val_loss: 0.1011 - val_accuracy: 0.9756 - 10s/epoch - 4ms/step
Epoch 8/100
2534/2534 - 10s - loss: 0.1130 - accuracy: 0.9642 - val_loss: 0.0989 - val_accuracy: 0.9747 - 10s/e

In [23]:
model_6.save('model1_6.h5')

In [39]:
# load models from file
# for both logistic and nueral
def load_all_models(n_models):
	all_models = list()

	for i in range(n_models):
		# define filename for this ensemble
#		colab_path = "gdrive/My Drive/Colab Notebooks/Network/models/"
		#model_4.save(colab_path + 'model4.h5')
		if i != 4 and i != 3:
			filename =  'model1_' + str(i + 1) + '.h5'
			# load model from file
			model = load_model(filename,custom_objects=None)
			# add to list of members
			all_models.append(model)
			print('>loaded %s' % filename)
	return all_models

In [40]:
# define stacked model from multiple member input models
#neural

def define_stacked_model(members):
	initializer = tf.keras.initializers.GlorotUniform(seed=64)
	constraints = None

  # update all layers in all models to not be trainable
	for i in range(len(members)):
		model = members[i]
		for layer in model.layers:
			# make not trainable
			layer.trainable = False
			# rename to avoid 'unique layer name' issue
			layer._name = 'ensemble_' + str(i + 1) + '_' + layer.name


	# define multi-headed input
	ensemble_visible = [model.input for model in members]
	# concatenate merge output from each model
	ensemble_outputs = [model.output for model in members]
	merge = concatenate(ensemble_outputs)
	hidden1 = Dense(128, kernel_initializer=initializer, activation='relu',name="dense_ea")(merge)
	hidden2 = Dense(64, kernel_initializer=initializer, activation='relu',name="dense_eb")(hidden1)
	output = Dense(Y.shape[1], kernel_initializer=initializer, activation='softmax',name="dense_ec")(hidden1)
	model = Model(inputs=ensemble_visible, outputs=output)
	# plot graph of ensemble
	plot_model(model, show_shapes=True, to_file='model_graph.png')
	# compile
	model.summary()
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [41]:
# fit a stacked model
#neural
import datetime


#colab_path = "gdrive/My Drive/Colab Notebooks/Network/models/"
log_dir =  "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

monitor_7= [
             tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
            ]

def fit_stacked_model(model, inputX, inputy):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# encode output data
	inputy_enc = to_categorical(inputy)
	# fit model
	history_7=model.fit(X, inputy_enc, epochs=60, verbose=2,callbacks=[monitor_7])

In [42]:
# make a prediction with a stacked model
#neural
def predict_stacked_model(model, inputX):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# make prediction
	return model.predict(X, verbose=2)

In [44]:
# load all models
#neural
from keras.models import load_model
n_members = 6
members = load_all_models(n_members)
print('Loaded %d models' % len(members))

>loaded model1_1.h5
>loaded model1_2.h5
>loaded model1_3.h5
>loaded model1_6.h5
Loaded 4 models


In [46]:
# define ensemble model
import matplotlib.pyplot as plt
#from keras.utils import plot_model
#neural
#from keras.layers.merge import concatenate
from keras.layers import concatenate
from keras.layers import Dense
from keras.models import Model
from tensorflow.keras.utils import plot_model

stacked_model = define_stacked_model(members)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 dense_2a_input (InputLayer)    [(None, 127)]        0           []                               
                                                                                                  
 dense_3a_input (InputLayer)    [(None, 127)]        0           []                               
                                                                                                  
 dense_6a_input (InputLayer)    [(None, 127)]        0           []                               
                                                                                                  
 dense_1a_input (InputLayer) 

In [47]:
from keras.utils import to_categorical
# fit stacked model on test dataset
#neural
fit_stacked_model(stacked_model, x_train_1, y_train_1)

Epoch 1/60
27250/27250 - 38s - loss: 0.0644 - accuracy: 0.9845 - 38s/epoch - 1ms/step
Epoch 2/60
27250/27250 - 36s - loss: 0.0557 - accuracy: 0.9850 - 36s/epoch - 1ms/step
Epoch 3/60
27250/27250 - 36s - loss: 0.0547 - accuracy: 0.9852 - 36s/epoch - 1ms/step
Epoch 4/60
27250/27250 - 37s - loss: 0.0547 - accuracy: 0.9850 - 37s/epoch - 1ms/step
Epoch 5/60
27250/27250 - 37s - loss: 0.0544 - accuracy: 0.9852 - 37s/epoch - 1ms/step
Epoch 6/60
27250/27250 - 37s - loss: 0.0541 - accuracy: 0.9851 - 37s/epoch - 1ms/step
Epoch 7/60
27250/27250 - 37s - loss: 0.0541 - accuracy: 0.9851 - 37s/epoch - 1ms/step
Epoch 8/60
27250/27250 - 37s - loss: 0.0541 - accuracy: 0.9852 - 37s/epoch - 1ms/step
Epoch 9/60
27250/27250 - 37s - loss: 0.0542 - accuracy: 0.9852 - 37s/epoch - 1ms/step
Epoch 10/60
27250/27250 - 37s - loss: 0.0541 - accuracy: 0.9852 - 37s/epoch - 1ms/step
Epoch 11/60
27250/27250 - 37s - loss: 0.0540 - accuracy: 0.9852 - 37s/epoch - 1ms/step
Epoch 12/60
27250/27250 - 37s - loss: 0.0543 - accur

In [48]:
from sklearn.metrics import accuracy_score
import numpy as np
# make predictions and evaluate
#neural
yhat = predict_stacked_model(stacked_model, x_test_1)
yhat_val = tf.argmax(yhat, axis=1)
acc = accuracy_score(y_test_1, yhat_val)
print('Stacked Test Accuracy: %.4f' % acc)


6813/6813 - 6s - 6s/epoch - 907us/step
Stacked Test Accuracy: 0.9841


In [49]:
from sklearn.metrics import classification_report
print(classification_report(y_test_1, yhat_val))
classification_report(y_test_1, yhat_val)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.92      0.96      0.94        51
           2       0.99      0.99      0.99     72091
           3       1.00      1.00      1.00       433
           4       1.00      0.97      0.98       365
           5       0.99      0.99      0.99     62503
           6       0.98      0.99      0.99     51385
           7       0.59      0.69      0.64        78
           8       0.46      0.31      0.37        67
           9       0.94      0.71      0.81        62
          10       0.91      0.21      0.34       349
          11       0.98      0.95      0.96      7418
          12       1.00      0.20      0.33         5
          13       1.00      0.67      0.80         9
          15       0.89      0.74      0.81      1989
          16       0.86      0.35      0.50        17
          17       0.91      0.94      0.92      3272
          18       1.00    

'              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00         3\n           1       0.92      0.96      0.94        51\n           2       0.99      0.99      0.99     72091\n           3       1.00      1.00      1.00       433\n           4       1.00      0.97      0.98       365\n           5       0.99      0.99      0.99     62503\n           6       0.98      0.99      0.99     51385\n           7       0.59      0.69      0.64        78\n           8       0.46      0.31      0.37        67\n           9       0.94      0.71      0.81        62\n          10       0.91      0.21      0.34       349\n          11       0.98      0.95      0.96      7418\n          12       1.00      0.20      0.33         5\n          13       1.00      0.67      0.80         9\n          15       0.89      0.74      0.81      1989\n          16       0.86      0.35      0.50        17\n          17       0.91      0.94      0.92      3272\n       